# Chatbot Demo 

## Description

This notebook demonstrates how to create a simple chatbot interface using Llama models via Hugging Face's `transformers` library and `gradio` for the user interface. 

### Main Features:
- Select from a list of pre-trained Llama models.
- Input text and receive responses from the selected model.
- Cache models to avoid reloading them multiple times.

## Log in to Hugging Face Hub

In this cell, we import the `login` function from the Hugging Face Hub and call it to authenticate with your Hugging Face account. This step is required to access private models, datasets, and other resources hosted on Hugging Face.

### Directions to Generate Access Token:
1. Go to the [Hugging Face website](https://huggingface.co/).
2. Log in to your Hugging Face account.
3. Navigate to your **profile icon** on the top right, and click **Settings**.
4. Under **Access Tokens** (on the left sidebar), click **New Token** to generate a new access token.
5. Copy the generated token.

### Usage:
When you run this cell, you'll be prompted to paste the access token, which grants access to your Hugging Face resources.

> **Do not share your Access Tokens with anyone**

In [ ]:
from huggingface_hub import login
login()

## Import Required Libraries

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
device = 0 if torch.cuda.is_available() else -1

## Basic Usage

Below there are a list of available Llama models to choose from. The function, `load_model`, loads the selected Llama model along with its tokenizer. It uses Hugging Face's `AutoModelForCausalLM` and `AutoTokenizer` to load the pre-trained model and return a text generation pipeline. Note that, for each version of Llama you need to seperately request access to be able to use them.

In [ ]:
llama_models = {
    "Llama 3 70B Instruct": "meta-llama/Meta-Llama-3-70B-Instruct",
    "Llama 3 8B Instruct": "meta-llama/Meta-Llama-3-8B-Instruct",
    "Llama 3.1 70B Instruct": "meta-llama/Llama-3.1-70B-Instruct",
    "Llama 3.1 8B Instruct": "meta-llama/Llama-3.1-8B-Instruct",
    "Llama 3.2 3B Instruct": "meta-llama/Llama-3.2-3B-Instruct",
    "Llama 3.2 1B Instruct": "meta-llama/Llama-3.2-1B-Instruct",
}

In [ ]:
def load_model(model_name):
    """Load the specified Llama model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=device)
    return generator

Cache models to avoid reloading

In [ ]:
model_cache = {}

This cell defines a `system_prompts` dictionary that provides specific instructions for different chatbot tasks:

- **Conversation**: A general-purpose helpful assistant.
- **Sentiment analysis**: An assistant that analyzes user input and assigns a sentiment label along with a brief explanation behind the reasoning.
- **Summarization**: An assistant that summarizes user input, focusing on key points and main ideas.

Each prompt includes a role ("system") and task-specific content to guide the chatbot's behavior during interactions.

In [ ]:
system_prompts = {"conversation": {
                        "role" : "system", 
                        "content" : "You are helpful assistant"
                  },
                  "sentiment analysis": {
                        "role": "system",
                        "content": "You are a helpful assistant specialized in sentiment analysis. Analyze the user's input and provide a sentiment label (positive, negative, or neutral) along with a brief explanation of your reasoning."
                  },
                  "summarization": {
                        "role": "system",
                        "content": "You are a helpful assistant specialized in summarizing text. Provide concise and coherent summaries by focusing on the key points and main ideas in the user input."
                  }}

The `generate_chat` function generates chatbot responses using a selected Llama model and task. It first checks if the model is cached, loading it if necessary. The appropriate system prompt is chosen based on the `task_choice` (e.g., "conversation", "summarization", or "sentiment_analysis"). The function builds the conversation context from the prompt and chat history, then adds the user's input. This context is passed to the model for response generation. The response is extracted, appended to the history, and the updated conversation history is returned.

In [ ]:
def generate_chat(user_input, history, model_choice, task_choice):
    """Generate chatbot responses using the selected Llama model and task."""
    if model_choice not in model_cache:
        model_cache[model_choice] = load_model(llama_models[model_choice])
    generator = model_cache[model_choice]

    system_prompt = system_prompts.get(task_choice.lower(), system_prompts["conversation"])

    messages = [system_prompt]

    if history is None:
        history = []

    for message in history:
        messages.append({"role": "user", "content": str(message[0])})
        messages.append({"role": "assistant", "content": str(message[1])})

    messages.append({"role": "user", "content": str(user_input)})

    response = generator(
        messages,
        max_length=512,
        pad_token_id=generator.tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )[-1]["generated_text"][-1]["content"]

    history.append((user_input, response))

    return history

The cell below defines a simple Gradio interface. The interface consists of two dropdown menus, one for users to select specific Llama model and the other to select the task, a textbox to enter queries and a window that displays the conversation between the user and the assistant.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Chat with Llama Models</center></h1>")
    model_choice = gr.Dropdown(list(llama_models.keys()), label="Select Llama Model")

    task_choice = gr.Dropdown(["Conversation", "Summarization", "Sentiment Analysis"], label="Select Task")

    chatbot = gr.Chatbot()
    txt_input = gr.Textbox(show_label=False, placeholder="Type your message here...")

    def respond(user_input, chat_history, model_choice, task_choice):
        updated_history = generate_chat(user_input, chat_history, model_choice, task_choice)
        return "", updated_history

    txt_input.submit(respond, [txt_input, chatbot, model_choice, task_choice], [txt_input, chatbot])

    submit_btn = gr.Button("Submit")
    submit_btn.click(respond, [txt_input, chatbot, model_choice, task_choice], [txt_input, chatbot])

Launch the interface

In [ ]:
demo.launch()